# Initial Setup

Configure GPU Hostage:

Edit → Notebook Settings → Hardware accelerator → (GPU)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Connect Google Drive

Connect your personal drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Copy pnml datasets into your personal drive and set the following paths

In [ ]:
path = "/content/drive/MyDrive/ilp-matcher/eval-data/csv/"

# Set up environment

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 636 kB 43.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install -U sentence_transformers

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 41.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=5f97981d4807d41acae3b182a66e051543d7c488b9836c23ecef721dda3a9ab8
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [ ]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')

#Define datasets to train model on
datasets = ["uni", "birth"]

#Combine datasets
combined_csv = pd.concat([pd.read_csv(path+ds+"-matches.csv") for ds in datasets ])
#export to csv
combined_csv.to_csv(path+"td-"+datasets[0]+"-"+datasets[1]+".csv", index=False, encoding='utf-8-sig')

In [ ]:
df = pd.read_csv(path+"td-"+datasets[0]+"-"+datasets[1]+".csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 703



,label1,label2,similarity
521,Give prename,Decide on first name 1,1
365,Confirm identity,Confirm identity,1
633,Create birth certificate,Sign certficate,1
87,Receive application,NaN,1
36,Take interview,Take interview,1
679,Send birth certificate to respective local pub...,Send PL,1
143,Decide first name,Decide on first name 1,1
501,Registration in one's place,Correct error,1
446,Check data,Check GBA,1
320,Inform responsible authority,Send notifications,1


In [ ]:
#Define your train examples. You need more than just two examples...
with open(path+"td-"+datasets[0]+"-"+datasets[1]+".csv") as csvdatei:
    csv_reader_object = csv.reader(csvdatei)
    train_examples = []

    count = 0
    for row in csv_reader_object:
        if count >= 1:
          train_examples.append(InputExample(texts=[row[0],row[1]], label=1.0))
          count += 1
        else:
          count += 1

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, output_path="/content/drive/MyDrive/Colab Notebooks/ilp-matcher/eval-data/bert-"+datasets[0]+"-"+datasets[1], save_best_model=True)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/44 [00:00<?, ?it/s]